
# DS6050 - Group 6
* Andrej Erkelens <wsw3fa@virginia.edu>
* Robert Knuuti <uqq5zz@virginia.edu>
* Khoi Tran <kt2np@virginia.edu>

## Abstract
English is a verbose language with over 69% redundancy in its construction, and as a result, individuals only need to identify important details to comprehend an intended message.
While there are strong efforts to quantify the various elements of language, the average individual can still comprehend a written message that has errors, either in spelling or in grammar.
The emulation of the effortless, yet obscure task of reading, writing, and understanding language is the perfect challenge for the biologically-inspired methods of deep learning.
Most language and text related problems rely upon finding high-quality latent representations to understand the task at hand. Unfortunately, efforts to overcome such problems are limited to the data and computation power available to individuals; data availability often presents the largest problem, with small, specific domain tasks often proving to be limiting.
Currently, these tasks are often aided or overcome by pre-trained large language models (LLMs), designed by large corporations and laboratories.
Fine-tuning language models on domain-specific vocabulary with small data sizes still presents a challenge to the language community, but the growing availability of LLMs to augment such models alleviates the challenge.
This paper explores different techniques to be applied on existing language models (LMs), built highly complex Deep Learning models, and investigates how to fine-tune these models, such that a pre-trained model is used to enrich a more domain-specific model that may be limited in textual data.

## Project Objective

We are aiming on using several small domain specific language tasks, particularly classification tasks.
We aim to take at least two models, probably BERT and distill-GPT2 as they seem readily available on HuggingFace and TensorFlow's model hub.
We will iterate through different variants of layers we fine tune and compare these results with fully trained models, and ideally find benchmarks already in academic papers on all of the datasets.

We aim to optimize compute efficiency and also effectiveness of the model on the given dataset. Our goal is to find a high performing and generalizable method for our fine tuning process and share this in our paper.


In [1]:
%autosave 0

Autosave disabled


In [2]:
!pip install -q tensorflow-text tokenizers transformers

In [3]:
import tensorflow as tf
import tensorflow_text as tf_text

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/ds6050/git/

/content/drive/MyDrive/ds6050/git


In [6]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import tokenizers
import torch
import transformers

from tensorflow import keras
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers

np.random.seed(42)
tf.random.set_seed(42)

df = pd.read_feather("data-extractor/data/dataset.feather")#.set_index('index')
df['topic'] = df['topic'].str.split('.').str[0]
df_train = df.sample(frac = 0.8)
df_test = df.drop(df_train.index)

In [7]:
features = 'content' # feature for the future - add all the datasets ['categories', 'summary', 'content']
label = 'topic'

In [8]:
# strategy = tf.distribute.MirroredStrategy()

In [9]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

# (45030, 7): 7 different topics
y_ = ohe.fit_transform(df['topic'].values.reshape(-1,1)).toarray()

In [10]:
#max_len = 512
checkpoint = 'gpt2'
hf_gpt2_tokenizer = transformers.GPT2Tokenizer.from_pretrained(checkpoint, add_prefix_space=True)
hf_gpt2_model = transformers.TFGPT2Model.from_pretrained(checkpoint)
# hf_gpt2_model = transformers.GPT2ForSequenceClassification.from_pretrained(checkpoint)

# add for gpt2 padding
if hf_gpt2_tokenizer.pad_token is None:
    hf_gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
hf_gpt2_model.resize_token_embeddings(len(hf_gpt2_tokenizer))

# batch encoding
encodings = hf_gpt2_tokenizer.batch_encode_plus(list(df.summary.values), 
                                                return_tensors='tf', 
                                                padding='max_length',
                                                #add_special_tokens=True,
                                                max_length=None,
                                                truncation=True)

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
Using pad_token, but it is not set yet.


In [11]:
train_encodings = hf_gpt2_tokenizer.batch_encode_plus(list(df_train.summary.values), 
                                                      return_tensors='tf', 
                                                      padding='max_length',
                                                      max_length=None,
                                                      truncation=True)

test_encodings  = hf_gpt2_tokenizer.batch_encode_plus(list(df_test.summary.values), 
                                                      return_tensors='tf', 
                                                      padding='max_length',
                                                      max_length=None,
                                                      truncation=True)

y_train = ohe.fit_transform(df_train['topic'].values.reshape(-1,1)).toarray()
y_test  = ohe.fit_transform(df_test['topic'].values.reshape(-1,1)).toarray()

In [12]:
def model_top(pretr_model):
  input_ids = tf.keras.Input(shape=(1024,), dtype='int32')
  attention_mask = tf.keras.Input(shape=(1024,), dtype='int32')

  output = pretr_model(input_ids = input_ids, attention_mask = attention_mask)
  # output = pretr_model([input_ids, attention_mask])
  #pooler_output = output[1]
  pooler_output = tf.keras.layers.AveragePooling1D(pool_size=1024)(output[0])
  flattened_output = tf.keras.layers.Flatten()(pooler_output)
  
  output = tf.keras.layers.Dense(32, activation='tanh')(flattened_output)
  output = tf.keras.layers.Dropout(0.2)(output)

  output = tf.keras.layers.Dense(7, activation='softmax')(output)
  model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [13]:
model = model_top(hf_gpt2_model)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 tfgpt2_model (TFGPT2Model)     TFBaseModelOutputWi  124440576   ['input_1[0][0]',                
                                thPastAndCrossAtten               'input_2[0][0]']                
                                tions(last_hidden_s                                               
                                tate=(None, 1024, 7                                           

In [15]:
model.layers

In [16]:
model.layers[2].trainable = False

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 tfgpt2_model (TFGPT2Model)     TFBaseModelOutputWi  124440576   ['input_1[0][0]',                
                                thPastAndCrossAtten               'input_2[0][0]']                
                                tions(last_hidden_s                                               
                                tate=(None, 1024, 7                                           

In [18]:
!nvidia-smi

Mon Aug  8 05:08:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |   2580MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
checkpoint_filepath = './tmp/checkpoint'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="auto",
)

In [20]:
history = model.fit([train_encodings['input_ids'], 
                     train_encodings['attention_mask']], 
                    y_train, 
                    validation_split=.2,
                    epochs=4,
                    batch_size=32,
                    callbacks=[model_checkpoint_callback, early_stopping_callback])

Epoch 1/4


ResourceExhaustedError: ignored

In [ ]:
#train_labels = df_train['topic']
#test_labels  = df_test['topic']

#train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
#                                                         train_labels))

#test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
#                                                        test_labels))

In [ ]:
#training_args.strategy.scope()

In [ ]:
# hf_gpt2_model.fit(train_dataset)

In [ ]:
#train_encodings['input_ids']

In [ ]:
#hf_gpt2_model.compile(optimizer='adam',
#                      loss='categorical_crossentropy',
#                      metrics=['acc'])

#hf_gpt2_model.fit(train_encodings['input_ids'])
#hf_gpt2_model.fit(train_dataset, epochs=4, validation_data=test_dataset)

In [ ]:
with training_args.strategy.scope():
  model = hf_gpt2_model

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### Data Preview

In [ ]:
for text, label in ds_train.take(5):
  print('Text')
  print(text)
  print('Label')
  print(label)

In [ ]:
"""## This is currently broken
max_len = 384
hf_gpt2_tokenizer_bootstrapper = transformers.BertTokenizer.from_pretrained("gpt2")
hf_gpt2_model = transformers.TFGPT2Model.from_pretrained("gpt2")

save_path = Path("data") / "models"
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
hf_gpt2_tokenizer_bootstrapper.save_pretrained(save_path)
hf_gpt2_model.save_pretrained(save_path)

# Load the fast tokenizer from saved file
gpt2_tokenizer = tokenizers.GPT2TokenizerFast(str(save_path/"vocab.txt"), lowercase=True)

def tf_hf_gpt2encode(features, label):
    x = gpt2_tokenizer.encode(tf.compat.as_str(features), add_special_tokens=True)
    y = gpt2_tokenizer.encode(tf.compat.as_str(label), add_special_tokens=True)
    return x, y

def tf_hf_gpt2encodeds(features, label):
    encode = tf.py_function(func=tf_hf_gpt2encode, inp=[features, label], Tout=[tf.int64, tf.int64])
    return encode

encoded_input = ds_train.batch(256).map(tf_hf_gpt2encodeds)
output = transformers.TFGPT2Model(config=transformers.PretrainedConfig.from_json_file(str(save_path/"config.json")))
hf_gpt2 = output(encoded_input)"""

In [ ]:
"""
files = [] # Need to explode train_ds to sep files

tokenizer = tokenizers.GPT2TokenizerFast(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
)

tokenizer.train(
    files,
    vocab_size=10000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##",
)

# Save the files
tokenizer.save_model(args.out, args.name)"""